##### Copyright 2019 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Decodifique arquivos DICOM de exames de imagem

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/dicom"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/dicom.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/dicom.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/dicom.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este tutorial mostra como usar `tfio.image.decode_dicom_image` no TensorFlow IO para decodificar arquivos DICOM usando o TensorFlow.

## Configuração e uso

#### Baixe a imagem DICOM

A imagem DICOM usada neste tutorial vem do [dataset NIH Chest X-ray](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest) (raios-X de tórax da NIH).

Esse dataset contém 100.000 imagens anonimizadas de raios-X do tórax no formato PNG, fornecido pelo Centro Clínico NIH e que pode ser baixado [neste link](https://nihcc.app.box.com/v/ChestXray-NIHCC).

O Google Cloud também fornece uma versão DICOM das imagens, disponível no [Cloud Storage](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest).

Neste tutorial, você vai baixar um arquivo de amostra do dataset no [repositório do GitHub](https://github.com/tensorflow/io/raw/master/docs/tutorials/dicom/dicom_00000001_000.dcm).

Observação: para ver mais informações sobre o dataset, confira a seguinte referência:

- Xiaosong Wang, Yifan Peng, Le Lu, Zhiyong Lu, Mohammadhadi Bagheri, Ronald Summers, ChestX-ray8: Hospital-scale Chest X-ray Database and Benchmarks on Weakly-Supervised Classification and Localization of Common Thorax Diseases, IEEE CVPR, páginas 3.462-3.471, 2017


In [ ]:
!curl -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/dicom/dicom_00000001_000.dcm
!ls -l dicom_00000001_000.dcm

### Instale os pacotes necessários e reinicie o runtime

In [ ]:
try:
  # Use the Colab's preinstalled TensorFlow 2.x
  %tensorflow_version 2.x 
except:
  pass

In [ ]:
!pip install tensorflow-io

### Decodifique a imagem DICOM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

In [ ]:
import tensorflow_io as tfio

image_bytes = tf.io.read_file('dicom_00000001_000.dcm')

image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)

skipped = tfio.image.decode_dicom_image(image_bytes, on_error='skip', dtype=tf.uint8)

lossy_image = tfio.image.decode_dicom_image(image_bytes, scale='auto', on_error='lossy', dtype=tf.uint8)


fig, axes = plt.subplots(1,2, figsize=(10,10))
axes[0].imshow(np.squeeze(image.numpy()), cmap='gray')
axes[0].set_title('image')
axes[1].imshow(np.squeeze(lossy_image.numpy()), cmap='gray')
axes[1].set_title('lossy image');

### Decodifique os metadados DICOM e trabalhe com tags

`decode_dicom_data` decodifica as informações de tags. `dicom_tags` contém informações úteis, como a idade e o sexo do paciente. Portanto, você pode usar tags DICOM como `dicom_tags.PatientsAge` e `dicom_tags.PatientsSex`. O tensorflow_io usa a mesma notação de tags do pacote DICOM pydicom.

In [ ]:
tag_id = tfio.image.dicom_tags.PatientsAge
tag_value = tfio.image.decode_dicom_data(image_bytes,tag_id)
print(tag_value)

In [ ]:
print(f"PatientsAge : {tag_value.numpy().decode('UTF-8')}")

In [ ]:
tag_id = tfio.image.dicom_tags.PatientsSex
tag_value = tfio.image.decode_dicom_data(image_bytes,tag_id)
print(f"PatientsSex : {tag_value.numpy().decode('UTF-8')}")

## Documentação

Esse pacote tem duas operações que encapsulam funções `DCMTK`. `decode_dicom_image` decodifica os dados de pixel dos arquivos DICOM, e `decode_dicom_data` decodifica as informações de tags. `tags` contém tags DICOM úteis, como `tags.PatientsName`. A notação de tags é a mesma do pacote DICOM [`pydicom`](https://pydicom.github.io/).

### Obtendo dados da imagem DICOM

```python
io.dicom.decode_dicom_image(
    contents,
    color_dim=False,
    on_error='skip',
    scale='preserve',
    dtype=tf.uint16,
    name=None
)
```

- **`contents`**: Tensor do tipo string. 0-D. Arquivo DICOM codificado com byte string
- **`color_dim`**: `bool` opcional. O padrão é `False`. Se `True`, um terceiro canal é anexado a todas as imagens, formando um tensor tridimensional. Uma imagem 1024x1024 em escala de cinza será 1024x1024x1
- **`on_error`**: o padrão é `skip` (ignorar). Esse atributo define o comportamento caso ocorra um erro ao abrir a imagem ou se o tipo de saída não conseguir acomodar todos os possíveis valores de entrada. Por exemplo, se o usuário definir o dtype de saída como `tf.uint8`, mas uma imagem DICOM armazenar um tipo `tf.uint16`, `strict` vai gerar um erro. `skip` retorna um tensor unidimensional vazio. `lossy` continua a operação, dimensionando o valor por meio do atributo `scale`.
- **`scale`**: o padrão é `preserve` (preservar). Esse atributo define o que fazer com a escala dos valores de entrada. `auto` fará o dimensionamento automático dos valores de entrada. Se o tipo de saída for inteiro, `auto` usará a escala de saída máxima, por exemplo, `uint8`, que armazena valores de [0, 255], para alargar linearmente para preencher um `uint16`, com valores [0,65535]. Se a saída for float, `auto` vai dimensionar no intervalo [0,1]. `preserve` mantém os valores como eles são. Um valor de entrada maior do que a saída máxima possível será recortado.
- **`dtype`**: um `tf.DType` opcional dentre: `tf.uint8, tf.uint16, tf.uint32, tf.uint64, tf.float16, tf.float32, tf.float64`. O padrão é `tf.uint16`.
- **`name`**: nome da operação (opcional).

**Retorna** um `Tensor` do tipo `dtype`, e o formato é determinado pelo arquivo DICOM.

### Obtendo os dados de tags DICOM

```python
io.dicom.decode_dicom_data(
    contents,
    tags=None,
    name=None
)
```

- **`contents`**: Tensor do tipo string. 0-D. Arquivo DICOM codificado com byte string
- **`tags`**: Tensor do tipo `tf.uint32` de qualquer dimensão. Esses números `uint32` mapeiam diretamente para tags DICOM
- **`name`**: nome da operação (opcional).

**Retorna** um `Tensor` do tipo `tf.string` e mesmo formato que `tags`. Se uma tag DICOM for uma lista de strings, elas são combinadas em uma string e separadas por uma barra invertida dupla `\`. Existe um bug em  [DCMTK](https://support.dcmtk.org/docs/): se a tag for uma lista de números, somente o elemento com índice 0 será retornado como uma string.

### Bibtex

Se esse pacote tiver ajudado, por favor, cite o seguinte:

```
@misc{marcelo_lerendegui_2019_3337331,
  author       = {Marcelo Lerendegui and
                  Ouwen Huang},
  title        = {Tensorflow Dicom Decoder},
  month        = jul,
  year         = 2019,
  doi          = {10.5281/zenodo.3337331},
  url          = {https://doi.org/10.5281/zenodo.3337331}
}
```

### Licença

Copyright 2019 Marcelo Lerendegui, Ouwen Huang, Gradient Health Inc.

Licenciado com Licença Apache, Versão 2.0 ("Licença"); você não pode usar este arquivo, exceto conforme disposto pela Licença. Você pode obter uma cópia da Licença em:

http://www.apache.org/licenses/LICENSE-2.0

A menos que exigido pelas leis aplicáveis ou acordado por escrito, o software distribuído com a Licença é distribuído "DA FORMA EM QUE SE ENCONTRA", SEM GARANTIAS OU CONDIÇÕES DE QUALQUER TIPO, explícitas ou implícitas. Confira a Licença para ver as informações específicas sobre permissões e limitações da Licença.